In [1]:
# Multinomial Classification 구현
# 사용하는 데이터 셋 = BMI 데이터셋
# tensorflow keras와 scikit-learn을 이용해서 구현해보아요!

In [18]:
# 일단 필요한 모듈부터 import를 해 보아요!
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [20]:
# Raw Data Loading
df = pd.read_csv('/content/drive/MyDrive/KOSA3/data/bmi/bmi.csv',
                 skiprows=3) # 4번째 행부터 불러와야하기 때문에 3번째 행까지 skip
display(df.head(), df.shape)

,label,height,weight
0,1,188,71
1,2,161,68
2,0,178,52
3,2,136,63
4,1,145,52


(20000, 3)

In [21]:
# 데이터 전처리



# 0. 데이터 자체에서 컬럼 정리
# 할거 없음


# 1. 결측치 처리 or 제거
df.info() # 결측치 없는 것을 확인
# (df.isnull()).sum() # 결측치를 수치로 확인


# 2. 이상치 제거
# plt.boxplot(df['label'].values) # label에는 이상치가 없다.
#plt.hist(df['label'].values, bins=3)
# plt.boxplot(df['height'].values) # height에는 이상치가 없다.
# plt.boxplot(df['weight'].values) # weight에는 이상치가 없다.
plt.show()
# => 만약 이상치가 있다면, zscore나 사분위값으로 이상치 처리를 해주어야 함


# 3. 정규화
x_data = df[['height', 'weight']].values # 2차원
t_data = df['label'] # 1차원

scaler = MinMaxScaler()
scaler.fit(x_data)

x_data_norm = scaler.transform(x_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   label   20000 non-null  int64
 1   height  20000 non-null  int64
 2   weight  20000 non-null  int64
dtypes: int64(3)
memory usage: 468.9 KB


In [22]:
# Scikit Learn 구현
# 1. scikit-learn을 위해 x_data 분할
#    train data 와 test data 분할

x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(x_data,
                 t_data,
                 test_size=0.3)

In [23]:
# 2. 모델 구현 후 학습 및 테스트
sklearn_model = linear_model.LogisticRegression()
sklearn_model.fit(x_data_train,
                  t_data_train)

sklearn_model_result = sklearn_model.score(x_data_test,
                                            t_data_test)
print('sklearn model의 accuracy : {}'.format(sklearn_model_result))
# sklearn model의 accuracy : 0.982

sklearn model의 accuracy : 0.9841666666666666


In [24]:
# 3. 예측
print(sklearn_model.predict_proba([[187, 62]]))
# [[9.96667326e-01 3.33267426e-03 1.14655717e-41]]
# 이 셋중에 가장 큰 값을 찾는 것 -> 가장 큰 확률값
# label이 1일 확률이 가장 크다(두번째 컬럼의 확률이 가장 크다)

[[9.97265604e-01 2.73439594e-03 3.62384304e-40]]


In [27]:
# Tensorflow Keras 구현

keras_model = Sequential()

keras_model.add(Flatten(input_shape=(2,)))    # 독립변수의 개수 2개
keras_model.add(Dense(units=3,                # units=3인 이유, label이 종속변수로 1일 확률/2일 확률/3일 확률 이렇게 3가지로 나누어지기 때문에
                      activation='softmax'))
keras_model.compile(optimizer=Adam(learning_rate=1e-2),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

## 정규화된 데이터를 사용해야한다
x_data_norm_train, x_data_norm_test, t_data_train, t_data_test = \
train_test_split(x_data_norm,
                 t_data,
                 test_size=0.3)


keras_model.fit(x_data_norm_train,
                t_data_train,
                epochs=200,
                verbose=1,
                validation_split=0.3)

Epoch 1/200
307/307 [==============================] - 3s 5ms/step - loss: 0.8687 - accuracy: 0.6164 - val_loss: 0.6921 - val_accuracy: 0.8319
Epoch 2/200
307/307 [==============================] - 1s 3ms/step - loss: 0.6049 - accuracy: 0.8307 - val_loss: 0.5367 - val_accuracy: 0.8295
Epoch 3/200
307/307 [==============================] - 0s 2ms/step - loss: 0.4947 - accuracy: 0.8782 - val_loss: 0.4549 - val_accuracy: 0.8962
Epoch 4/200
307/307 [==============================] - 0s 1ms/step - loss: 0.4280 - accuracy: 0.9126 - val_loss: 0.4011 - val_accuracy: 0.9267
Epoch 5/200
307/307 [==============================] - 0s 2ms/step - loss: 0.3815 - accuracy: 0.9332 - val_loss: 0.3601 - val_accuracy: 0.9436
Epoch 6/200
307/307 [==============================] - 0s 2ms/step - loss: 0.3464 - accuracy: 0.9509 - val_loss: 0.3290 - val_accuracy: 0.9579
Epoch 7/200
307/307 [==============================] - 0s 2ms/step - loss: 0.3181 - accuracy: 0.9610 - val_loss: 0.3049 - val_accuracy: 0.9521

In [28]:
# 학습이 끝났으므로 평가를 해야한다
print(keras_model.evaluate(x_data_norm_test, t_data_test)) # [0.061996765434741974, 0.9821666479110718]
## 즉, 정확도는 약 98.21%

188/188 [==============================] - 0s 1ms/step - loss: 0.0620 - accuracy: 0.9822
[0.061996765434741974, 0.9821666479110718]


In [29]:
# Prediction

# 예측할 때, 정규화된 데이터들을 넣었기 때문에
# 예측할 때도 정규화된 데이터들을 넣어주어야 한다.
my_state = np.array([[187, 82]])
my_state_norm = scaler.transform(my_state)
print(keras_model.predict(my_state_norm))
## [[6.6434858e-08 9.3651336e-01 6.3486509e-02]]
## 셋 중에 가장 큰 값을 가진 Label로 분류될 것이다

1/1 [==============================] - 0s 81ms/step
[[6.6434858e-08 9.3651336e-01 6.3486509e-02]]
